# Leave only those sentences pr PMIDs that have the food or compound name

Use Jaro-Winkler distance and ngrams to filter the sentences

Depends on hits and pmids pickle files that are created by download_food_ids.ipynb.

In [20]:
from __future__ import print_function
import sys,pickle, re, jellyfish
import pubmed.utils as pb
from collections import defaultdict
from nltk.tokenize import sent_tokenize
from urllib2 import HTTPError

# load temporary results 
hits0 = pickle.load(open( "hits_0_3642.pickle", "rb" ) )
pmids0 = pickle.load(open( "pmids_0_3642.pickle", "rb" ) )
hits1 = pickle.load(open( "hits_3643_3799.pickle", "rb" ) )
pmids1 = pickle.load(open( "pmids_3643_3799.pickle", "rb" ) )
hits2 = pickle.load(open( "hits_3799_end.pickle", "rb" ) )
pmids2 = pickle.load(open( "pmids_3799_end.pickle", "rb" ) ) 

In [2]:

def splitSentences(abstract):
    sentences = sent_tokenize(abstract)
    return sentences

def flat_map(sentences):
    return [sent for s in sentences for sent in s]

def findFood(sentence, foods):
    ''' Calculates the Jaro Wrinkler distance between food name and ngrams in the sentence.
        Returns True if distance > limit
    '''
 
    result = False
    # If using string distance method:
    for f in foods:
        n = min(3, len(f.split()))  # Assuming max as trigram        
        sentence_ngrams = find_ngrams(sentence, n)  # Note: punctuation at end of sentence will be included with
                                                    # last word. For now ok, since the JW will still be > 05
        for ngram in sentence_ngrams:
            # Note: when using jaro_winkler, need to convert into unicode format
            if jellyfish.jaro_winkler(f.lower(), u"{}".format(ngram.lower())) > 0.95:
                result = True
                break
    return result

def findDrug(sentence, dglist, limit):
    ''' Calculates the Jaro Wrinkler distance between drug name and ngrams in the sentence.
        Returns True if distance > limit
    '''
    result = False
    for drug in dglist:
        n = min(5, len(drug.split()))  # Assuming max as 5-gram        
        sentence_ngrams = find_ngrams(sentence, n)  # Note: punctuation at end of sentence will be included with
                                                # last word. For now ok, since the JW will still be > limit
        for ngram in sentence_ngrams:
            # Note: when using jaro_winkler, need to convert into unicode format
            ngram_distance = jellyfish.jaro_winkler(u"{}".format(drug.lower()), u"{}".format(ngram.lower()))
            if ngram_distance > limit:  
                result = True
                break

    return result


def find_ngrams(sentence, n):
    ''' Return list of ngrams from a sentence
    '''
    words_list = sentence.split()
    ngrams = zip(*[words_list[i:] for i in range(n)])
    return [''.join([w+' ' for w in ngram]).strip() for ngram in ngrams]



## Build food - id - sentence dictionary
Extract only those that have fewer than 100 ids (more than those are likely not valid results)

In [21]:
foods_pmids = defaultdict(list)
# food names are between quotation marks in the search term
RE = re.compile(r'"(.*?)"')
for hits, pmids in zip([hits0, hits1, hits2],[pmids0, pmids1, pmids2]):
    for fc, pmid in zip(hits, pmids):
        if len(pmid) < 100:
            food_or_compound = RE.findall(fc)[0]
            foods_pmids[food_or_compound] = pmid

Find sentences that contain both food names and ACE

In [45]:
filtered_sentences = defaultdict(lambda : defaultdict(list))
N_FOODS = len(foods_pmids)

for i, items in enumerate(foods_pmids.items()):
    food, pmids = items
    # start dynamic iteration counter to track where we are 
    print('\rFood or compound item:', i+1, '/', N_FOODS,end='')
    sys.stdout.flush()
    try:
        abstracts = pb.PubMedQuery.abstract_getter(pmids)
        sentences = [splitSentences(abstract) for abstract in abstracts.values()]
        for pmid, sents in zip(pmids, sentences):
            food_filtered = filter(lambda sent: findFood(sent, [unicode(food)]), sents)
            food_drug_filtered = filter(lambda sent: findDrug(sent, [u'angiotensin-converting enzyme',u'ACE'],0.8), 
                                        food_filtered)
            if len(food_drug_filtered) > 0:
                filtered_sentences[food][pmid] = food_drug_filtered
    except HTTPError:
        print('HTTPError here but why?')
    except UnicodeDecodeError:
        print(' Unicode again! How did it get here?')
    except TypeError:
        print(" Oh-no, a None, let's escape!")

   


Food or compound item: 2 / 1287 Unicode again! How did it get here?
Food or compound item: 145 / 1287 Oh-no, a None, let's escape!
Food or compound item: 207 / 1287 Unicode again! How did it get here?
Food or compound item: 486 / 1287 Unicode again! How did it get here?
Food or compound item: 540 / 1287 Unicode again! How did it get here?
Food or compound item: 711 / 1287 Unicode again! How did it get here?
Food or compound item: 1076 / 1287 Unicode again! How did it get here?
Food or compound item: 1132 / 1287 Unicode again! How did it get here?
Food or compound item: 1199 / 1287 Unicode again! How did it get here?
Food or compound item: 1276 / 1287 Unicode again! How did it get here?
Food or compound item: 1284 / 1287 Unicode again! How did it get here?
Food or compound item: 1287 / 1287

In [34]:
filtered_sentences[u'ginger'][u'23875904']

[u"Rubra) and white ginger (Zingiber officinale Roscoe) were prepared and the ability of the extracts to inhibit ACE along with Fe(2+)- and SNP-induced lipid peroxidation was determined in rat's heart in vitro.",
 u'However, red ginger extract (EC50=27.5 \u03bcg/mL) had a significantly (P<.05) higher inhibitory effect on ACE than white ginger extract (EC50=87.0 \u03bcg/mL).',
 u'This suggests that the possible mechanism through which ginger exerts its antihypertensive properties may be through inhibition of ACE activity and prevention of lipid peroxidation in the heart.',
 u'Furthermore, red ginger showed stronger inhibition of ACE than white ginger.']

Asssign to a new dictionary because pickle had problems with defaultdict lambda

In [41]:
new_filtered_sentences = {}
for key, value in filtered_sentences.items():
    new_filtered_sentences[key] = value

In [43]:
pickle.dump(new_filtered_sentences, open('ACE_filtered_sentences.pickle','wb'))

In [47]:
filtered_sentences

defaultdict(<function __main__.<lambda>>,
            {'1-octanol': defaultdict(list,
                         {u'19720379': ['Angiotensin 1, angiotensin 2, and angiotensin 3 migrated from 500 microL of diluted plasma, through a thin layer of 1-octanol and 8% di-(2-ethylhexyl) phosphate immobilized as a supported liquid membrane (SLM) in the pores of a porous hollow fiber, and into a 25 microL aqueous acceptor solution present inside the lumen of the fiber.']}),
             '12-hete': defaultdict(list,
                         {u'8621209': ['Acute blood pressure reduction attained during lipoxygenase inhibition but not by angiotensin converting enzyme inhibition leads to a concomitant reduction in the production of platelet 12-HETE.']}),
             '2-phenylethanol': defaultdict(list,
                         {u'19016485': ['2-Phenylethanol is a widely used aroma compound with rose-like fragrance and L-homophenylalanine is a building block of angiotensin-converting enzyme (ACE) inhi